<a href="https://colab.research.google.com/github/gulabpatel/simpletransformers/blob/main/Intro_to_Weights_%26_Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

# Quickstart
Use Weights & Biases for machine learning experiment tracking, dataset versioning, and project collaboration.

<div><img /></div>

<img src="https://i.imgur.com/uEtWSEb.png" width="650" alt="Weights & Biases" />

<div><img /></div>

It's also easy to download all your data and export it to other tools— like custom analysis in a Jupyter notebook. Here's more on our [API](https://docs.wandb.com/library/api).




Start by installing the library and logging in to your free account.

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

## Run W&B 
1. Start a new run, and pass in hyperparameters to track
2. Log metrics from training or evaluation
3. Visualize results in the dashboard

In [ ]:
import math
import random

# Start a new run, tracking hyperparameters in config
wandb.init(project="test-drive", config={
    "learning_rate": 0.01,
    "dropout": 0.2,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
})
config = wandb.config

# Simulating a training or evaluation loop
for x in range(50):
  acc = math.log(1 + x + random.random()*config.learning_rate) + random.random() + config.dropout
  loss = 10 - math.log(1 + x + random.random() + config.learning_rate*x) + random.random() + config.dropout
  # Log metrics from your script to W&B
  wandb.log({"acc":acc, "loss":loss})

Here's an example of what an interactive dashboard looks like in W&B.
![](https://i.imgur.com/4I51OuT.png)

# Example Model
Train a quick model in Keras and track results with W&B. Check our [examples repo](https://github.com/wandb/examples) for scripts using Keras, PyTorch, TensorFlow, Scikit, XGBoost and more.

## 1- Simple Keras CNN
Run this model to train a simple MNIST classifier, and click on the project page link to see your results stream in live to a W&B project.

In [ ]:
# Simple Keras Model

import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from wandb.keras import WandbCallback

# Set an experiment name to group training and evaluation
experiment_name = wandb.util.generate_id()

# Start a run, tracking hyperparameters
wandb.init(
  project="intro-demo",
  group=experiment_name,
  config={
    "layer_1": 512,
    "activation_1": "relu",
    "dropout": 0.2,
    "layer_2": 10,
    "activation_2": "softmax",
    "optimizer": "sgd",
    "loss": "sparse_categorical_crossentropy",
    "metric": "accuracy",
    "epoch": 6,
    "batch_size": 32
  })
config = wandb.config

# Get the data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build a model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
    tf.keras.layers.Dropout(config.dropout),
    tf.keras.layers.Dense(config.layer_2, activation=config.activation_2)
])
model.compile(optimizer=config.optimizer,
              loss=config.loss,
              metrics=[config.metric]
              )

history = model.fit(x=x_train,
                    y=y_train,
                    epochs=config.epoch,
                    batch_size=config.batch_size,
                    validation_data=(x_test, y_test),
                    # Use the WandbCallback to automatically save all the metrics tracked in model.fit() to your dashboard
                    callbacks=[WandbCallback()])

## 2- Log Analysis

Next, log an analysis run, using the same experiment name as the `group` parameter so that this run and the previous run are grouped together in W&B.

In [ ]:
!pip install scikit-plot -qqq

import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_confusion_matrix, plot_roc

wandb.init(project="intro-demo", group=experiment_name)

# Log F1 Score
y_test_pred = np.asarray(model.predict(x_test))
y_test_pred_class = np.argmax(y_test_pred, axis=1)
f1 = f1_score(y_test, y_test_pred_class, average='micro')
wandb.log({"f1": f1}, commit=False)

# Log Confusion Matrix
fig, ax = plt.subplots(figsize=(16, 12))
plot_confusion_matrix(y_test, y_test_pred_class, ax=ax)
wandb.log({"confusion_matrix": wandb.Image(fig)}, commit=False)

# Log ROC Curve
fig, ax = plt.subplots(figsize=(16, 12))
plot_roc(y_test, y_test_pred, ax=ax)
wandb.log({"plot_roc": wandb.Image(fig)}, commit=True)

class_score_data = []
for test, pred in zip(y_test, y_test_pred):
  class_score_data.append([test, pred])
wandb.log({"class_scores": wandb.Table(data=class_score_data,
                                           columns=["test", "pred"])})

In [ ]:
print(y_test)
print(y_test_pred)

## 3- Visualize Results

Click on the **project page** link above to see your live results.

## 4- Example Gallery

See examples of projects tracked and visualized with W&B in the [Gallery →](https://app.wandb.ai/gallery)

# Best Practices

1. **Projects**: Log multiple runs to a project to compare them. `wandb.init(project="project-name")`
2. **Groups**: For multiple processes or cross validation folds, log each process as a runs and group them together. `wandb.init(group='experiment-1')`
3. **Tags**: Add tags to track your current baseline or production model.
4. **Notes**: Type notes in the table to track the changes between runs.
5. **Reports**: Take quick notes on progress to share with colleagues and make dashboards and snapshots of your ML projects.

### Advanced Setup
1. [Environment variables](https://docs.wandb.com/library/environment-variables): Set API keys in environment variables so you can run training on a managed cluster.
2. [Offline mode](https://docs.wandb.com/library/technical-faq#can-i-run-wandb-offline): Use `dryrun` mode to train offline and sync results later.
3. [On-prem](https://docs.wandb.com/self-hosted): Install W&B in a private cloud or air-gapped servers in your own infrastructure. We have local installations for everyone from academics to enterprise teams.
4. [Sweeps](https://docs.wandb.com/sweeps): Set up hyperparameter search quickly with our lightweight tool for tuning.
5. [Artifacts](https://docs.wandb.com/artifacts): Track and version models and datasets in a streamlined way that automatically picks up your pipeline steps as you train models.